In [907]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

In [908]:
data = yf.download('BTC-USD')

[*********************100%***********************]  1 of 1 completed


In [909]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2024-08-22,61168.316406,61408.109375,59815.253906,60381.914062,60381.914062,27625734377
2024-08-23,60380.953125,64947.062500,60372.050781,64094.355469,64094.355469,42530509233
2024-08-24,64103.871094,64513.789062,63619.917969,64178.992188,64178.992188,21430585163


In [910]:
df = data['Adj Close'].pct_change() * 100

In [911]:
df

Date
2014-09-17         NaN
2014-09-18   -7.192558
2014-09-19   -6.984265
2014-09-20    3.573492
2014-09-21   -2.465854
                ...   
2024-08-22   -1.296730
2024-08-23    6.148267
2024-08-24    0.132050
2024-08-25    0.240812
2024-08-27   -1.995011
Name: Adj Close, Length: 3632, dtype: float64

In [912]:
df = df.rename("Today")

In [913]:
df = df.reset_index()

In [914]:
df

,Date,Today
0,2014-09-17,NaN
1,2014-09-18,-7.192558
2,2014-09-19,-6.984265
3,2014-09-20,3.573492
4,2014-09-21,-2.465854
...,...,...
3627,2024-08-22,-1.296730
3628,2024-08-23,6.148267
3629,2024-08-24,0.132050
3630,2024-08-25,0.240812


In [915]:
for i in range(1,6):
     df['Lag ' + str(i)] = df['Today'].shift(i)

In [916]:
df

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
0,2014-09-17,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-09-18,-7.192558,NaN,NaN,NaN,NaN,NaN
2,2014-09-19,-6.984265,-7.192558,NaN,NaN,NaN,NaN
3,2014-09-20,3.573492,-6.984265,-7.192558,NaN,NaN,NaN
4,2014-09-21,-2.465854,3.573492,-6.984265,-7.192558,NaN,NaN
...,...,...,...,...,...,...,...
3627,2024-08-22,-1.296730,3.664288,-0.807921,1.726094,-1.672873,0.993083
3628,2024-08-23,6.148267,-1.296730,3.664288,-0.807921,1.726094,-1.672873
3629,2024-08-24,0.132050,6.148267,-1.296730,3.664288,-0.807921,1.726094
3630,2024-08-25,0.240812,0.132050,6.148267,-1.296730,3.664288,-0.807921


In [917]:
print(df.columns)

Index(['Date', 'Today', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5'], dtype='object')


In [918]:
df['Volume'] = data.Volume.shift(1).values/1000_000_000

In [919]:
df = df.dropna()

In [920]:
df

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume
6,2014-09-23,8.364742,0.835210,-2.465854,3.573492,-6.984265,-7.192558,0.024128
7,2014-09-24,-2.888081,8.364742,0.835210,-2.465854,3.573492,-6.984265,0.045100
8,2014-09-25,-2.748309,-2.888081,8.364742,0.835210,-2.465854,3.573492,0.030628
9,2014-09-26,-1.736994,-2.748309,-2.888081,8.364742,0.835210,-2.465854,0.026814
10,2014-09-27,-1.212833,-1.736994,-2.748309,-2.888081,8.364742,0.835210,0.021461
...,...,...,...,...,...,...,...,...
3627,2024-08-22,-1.296730,3.664288,-0.807921,1.726094,-1.672873,0.993083,32.731154
3628,2024-08-23,6.148267,-1.296730,3.664288,-0.807921,1.726094,-1.672873,27.625734
3629,2024-08-24,0.132050,6.148267,-1.296730,3.664288,-0.807921,1.726094,42.530509
3630,2024-08-25,0.240812,0.132050,6.148267,-1.296730,3.664288,-0.807921,21.430585


In [921]:
df = sm.add_constant(df)

In [922]:
df['Direction'] = [1 if i > 0 else 0 for i in df['Today']]

In [923]:
df

,const,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume,Direction
6,1.0,2014-09-23,8.364742,0.835210,-2.465854,3.573492,-6.984265,-7.192558,0.024128,1
7,1.0,2014-09-24,-2.888081,8.364742,0.835210,-2.465854,3.573492,-6.984265,0.045100,0
8,1.0,2014-09-25,-2.748309,-2.888081,8.364742,0.835210,-2.465854,3.573492,0.030628,0
9,1.0,2014-09-26,-1.736994,-2.748309,-2.888081,8.364742,0.835210,-2.465854,0.026814,0
10,1.0,2014-09-27,-1.212833,-1.736994,-2.748309,-2.888081,8.364742,0.835210,0.021461,0
...,...,...,...,...,...,...,...,...,...,...
3627,1.0,2024-08-22,-1.296730,3.664288,-0.807921,1.726094,-1.672873,0.993083,32.731154,0
3628,1.0,2024-08-23,6.148267,-1.296730,3.664288,-0.807921,1.726094,-1.672873,27.625734,1
3629,1.0,2024-08-24,0.132050,6.148267,-1.296730,3.664288,-0.807921,1.726094,42.530509,1
3630,1.0,2024-08-25,0.240812,0.132050,6.148267,-1.296730,3.664288,-0.807921,21.430585,1


In [924]:
X = df[['const','Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume']]

In [925]:
y = df.Direction 

In [926]:
model = sm.Logit(y, X)

In [927]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.689034
         Iterations 4


In [928]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Direction   No. Observations:                 3626
Model:                          Logit   Df Residuals:                     3619
Method:                           MLE   Df Model:                            6
Date:                Tue, 27 Aug 2024   Pseudo R-squ.:                0.003568
Time:                        09:08:44   Log-Likelihood:                -2498.4
converged:                       True   LL-Null:                       -2507.4
Covariance Type:            nonrobust   LLR p-value:                  0.006508
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1863      0.046      4.084      0.000       0.097       0.276
Lag 1         -0.0313      0.009     -3.376      0.001      -0.049      -0.013
Lag 2         -0.0053      0.009     -0.573      0.566      -0.023       0.013
Lag 3          0.0111      0.009      1.213      0.225      -0.007       0.029
Lag 4          0.0008      0.009      0.089      0.929      -0.017       0.019
Lag 5         -0.0054      0.009     -0.594      0.553      -0.023       0.013
Volume        -0.0037      0.002     -2.105      0.035      -0.007      -0.000
==============================================================================
"""

In [929]:
predictions = result.predict(X)

In [930]:
predictions    

6       0.561232
7       0.483345
8       0.554918
9       0.597365
10      0.556077
          ...   
3627    0.491876
3628    0.526780
3629    0.468642
3630    0.515933
3631    0.538922
Length: 3626, dtype: float64

In [931]:
y

6       1
7       0
8       0
9       0
10      0
       ..
3627    0
3628    1
3629    1
3630    1
3631    0
Name: Direction, Length: 3626, dtype: int64

In [932]:
def confusion_matrix(act, pred):
     predtrans = ['Up' if i > 0.5 else 'Down' for i in pred]
     actstrans = ['Up' if i > 0 else 'Down' for i in act]
     confusion_matrix = pd.crosstab(pd.Series(actstrans), pd.Series(predtrans), rownames=['Actual'], colnames=['Predicted'])
     return confusion_matrix

In [933]:
confusion_matrix(y, predictions)

Predicted,Down,Up
Actual,,
Down,302,1407
Up,275,1642


In [934]:
len(df)

3626

In [935]:
(788+1129)/3626

0.5286817429674573

In [936]:
print(df.columns)

Index(['const', 'Date', 'Today', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5',
       'Volume', 'Direction'],
      dtype='object')


In [937]:
# Adjust the year filter to include years present in the data
x_train = df[df.Date.dt.year <= 2019][['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume']]
y_train = df[df.Date.dt.year <= 2019]['Direction']
x_test = df[df.Date.dt.year == 2019][['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume']]
y_test = df[df.Date.dt.year == 2019]['Direction']



model = sm.Logit(y_train, x_train)

In [938]:
model = sm.Logit(y_train, x_train)

In [939]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.686297
         Iterations 4


In [940]:
predictions = result.predict(x_test)

In [941]:
confusion_matrix(y_test, predictions)

Predicted,Down,Up
Actual,,
Down,43,131
Up,54,137


In [942]:
(43+137)/len(x_test)

0.4931506849315068